In [2]:
import torch
import numpy
import torch.nn as nn
import matplotlib.pyplot as plt
import torch
%matplotlib inline

In [8]:
from utils import load_mnist

In [4]:
device =torch.device('cuda')  if torch.cuda.is_available() else torch.device('cpu')

In [5]:
d = torch.load('./model.pth',map_location=device)
model = d['model']
config = d['config']

In [9]:
test_x,test_y = load_mnist(is_train=False,flatten=True)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [12]:
print("|Test_x| :",test_x.shape)

|Test_x| : torch.Size([10000, 784])


In [13]:
def show_image(x):
    if x.dim() == 1:
        x = x.reshape(int(x.shape[0]**.5),-1)

    plt.imshow(x,cmap='gray')
    plt.show()